In [585]:
from bs4 import BeautifulSoup, SoupStrainer, BeautifulStoneSoup
import requests
import pandas as pd
import re

In [586]:
# Returns Dataframe of document links for a given CIK
def get_sec_docs(cik="0001065088"):
    base_url = "https://www.sec.gov/cgi-bin/browse-edgar"
    inputted_cik = cik
    payload = {
        "action" : "getcompany",
        "CIK" : inputted_cik,
        "type" : "8-K",
        "output":"xml"
        #"dateb" : "20180331",
        #"count" : "100",
        #"owner" : "include"
    }
    sec_response = requests.get(url=base_url,params=payload)
    soup = BeautifulSoup(sec_response.text,'lxml')
    url_list = soup.findAll('filinghref')
    html_list = []
    # Get html version of links
    for link in url_list:
        link = link.string
        if link.split(".")[len(link.split("."))-1] == 'htm':
            txtlink = link + "l"
            html_list.append(txtlink)

    doc_list = []
    doc_name_list = []
    # Get links for txt versions of files
    for k in range(len(html_list)):
        txt_doc = html_list[k].replace("-index.html",".txt")
        doc_name = txt_doc.split("/")[-1]
        doc_list.append(txt_doc)
        doc_name_list.append(doc_name)
        # Create dataframe of CIK, doc name, and txt link
    df = pd.DataFrame(
        {
        "cik" : [cik]*len(html_list),
        "txt_link" : doc_list,
        "doc_name": doc_name_list
        }
    )
    return df

In [587]:
# Extracts text and submission datetime from document link
def extract_text(link):
    r = requests.get(link)
    #Parse 8-K document
    filing = BeautifulSoup(r.content,"html.parser",from_encoding="ascii")
    #Extract datetime
    submission_dt = filing.find("acceptance-datetime").string[:14]
    #Extract HTML sections
    for section in filing.findAll("html"):
        #Remove tables
        for table in section("table"):
            table.decompose()
        #Convert to unicode
        section = unicodedata.normalize("NFKD",section.text)
        section = section.replace("\t"," ").replace("\n"," ").replace("/s"," ").replace("\'","'")
    filing = "".join((section))
    
    return filing, submission_dt

In [588]:
ebay_df = get_sec_docs()

In [589]:
corpus = extract_text(ebay_df['txt_link'][1])

In [590]:
corpus

('    Exhibit   Exhibit 99.1eBay Inc. Reports Fourth Quarter and Full Year 2017 ResultseBay Delivers Record Fourth Quarter Driven by Strong Holiday PerformanceSan Jose, California, January 31, 2018 - eBay Inc. (NASDAQ: EBAY), a global commerce leader, delivered revenue for the quarter ended December 31, 2017 of $2.6 billion, increasing 9% on an as-reported basis and 7% on a foreign exchange (FX) neutral basis, primarily driven by gross merchandise volume (GMV) of $24.4 billion, up 10% on an as-reported basis and 7% on an FX-Neutral basis. During the quarter, eBay delivered GAAP net loss from continuing operations of $2.6 billion, or $(2.51) per diluted share primarily driven by a $3.1 billion tax charge attributable to the enactment of the Tax Cuts and Jobs Act. Non-GAAP net income from continuing operations was $618 million, or $0.59 per diluted share. The company generated $988 million of operating cash flow and $796 million of free cash flow from continuing operations while also rep